## Libraries and Packages

In [1]:
# importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

C:\Python\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### 1.1 Looking for missing values

In [4]:
#join the train and test dataset

#Create the source column to seperate the data
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index=True)
print(train.shape,test.shape,data.shape)

(8523, 13) (5681, 12) (14204, 13)


In [5]:
#missing values in percentage
data.isnull().sum()/data.shape[0] *100

Item_Identifier               0.000000
Item_Weight                  17.171219
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.273726
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales            39.995776
source                        0.000000
dtype: float64

Observation :

     -  Item_Outlet_Sales is target variable and have missing values as test data dont have target variable
     -  Item_Weight is float variable and can be treated as either with mean or median
     -  Outlet_Size is categorical variable and can be imputed using mode

### 1.1 Imputing mean for Item_Weight missing values

In [6]:
print("Item_weight missing value before : ",data['Item_Weight'].isnull().sum())
data['Item_Weight'].fillna(data['Item_Weight'].mean(),inplace=True)
print("Item_weight missing value after : ",data['Item_Weight'].isnull().sum())

Item_weight missing value before :  2439
Item_weight missing value after :  0


### 1.2 Imputing mode for Outlet_Size missing values

In [7]:
print("Outlet_Size missing value before : ",data['Outlet_Size'].isnull().sum())
data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0],inplace=True)
print("Outlet_Size missing value after : ",data['Outlet_Size'].isnull().sum())

Outlet_Size missing value before :  4016
Outlet_Size missing value after :  0


## 2. Feature Engineering

### 2.1 Outlet_Type

In [8]:
#create pivot table for Outlet_Type with mean of Item_Outlet_Sales. aggfunc by default mean

data.pivot_table(values='Item_Outlet_Sales',columns='Outlet_Type')

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Item_Outlet_Sales,339.8285,2316.181148,1995.498739,3694.038558


Oservation : We cant combine Outlet_Type as they have different means

### 2.2 Item_visibility

min Item_visibility is equal to zero is not possible. lets consider it as missing value and impute it with mean

In [9]:
#Determine avg visibility of product

visibility_avg = data.pivot_table(values='Item_Visibility',index='Item_Identifier')

#imput 0 values with mean visibility of prod

missing_values = data['Item_Visibility']==0

print("Number of 0 values initially : %d" %sum(missing_values))

data.loc[missing_values,'Item_Visibility'] = data.loc[missing_values,'Item_Identifier'].apply(lambda x :
                                             visibility_avg.at[x,'Item_Visibility'])
                                                                                              
print("Number of 0 values after modification : %d" %sum(data['Item_Visibility']==0))

Number of 0 values initially : 879
Number of 0 values after modification : 0


### 2.3 Determine years of operation of store

Data is from year 2013, keeping that in mind, we will create new feature as store age

In [10]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

### 2.4 Create broad category of Item_Type

Item_Type have 16 categories, if we combine using Item_Identifier with FD:Foods, DR: Drinks and NC:Non-Consumables. we can make it into 3 categories

In [11]:
#get the first 2 letters of ID

data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x : x[0:2] )

#Renaming them 
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food','NC':'Non-Consumables','DR':'Drinks'})
data["Item_Type_Combined"].value_counts()

Food               10201
Non-Consumables     2686
Drinks              1317
Name: Item_Type_Combined, dtype: int64

### 2.5 Modify categories : Item_Fat_Content

In [12]:
#Correcting the typos in this category

print("Original Categories :")
print(data['Item_Fat_Content'].value_counts())

mapping = {
    'Low Fat' : 'Low Fat',
    'LF' : 'Low Fat',
    'low fat' : 'Low Fat',
    'reg' : 'Regular',
    'Regular' : 'Regular'
}

print("\n Modified Categories :")
data['Item_Fat_Content'] = data['Item_Fat_Content'].map(mapping)
print(data['Item_Fat_Content'].value_counts())

Original Categories :
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

 Modified Categories :
Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


Some of the items are Non consumable and still contains Fat. we will create new feature for the same

In [13]:
#Mark Non consumable to have separate category in Item_Fat_Content

data.loc[data['Item_Type_Combined'] == "Non-Consumables","Item_Fat_Content"] = "Non-Edible"

print(data['Item_Fat_Content'].value_counts())

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64


In [14]:
data.to_csv("processed_data.csv",index=False)

In [15]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Outlet_Years,Item_Type_Combined
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train,14,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train,4,Drinks
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train,14,Food
3,FDX07,19.20,Regular,0.017834,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,train,15,Food
4,NCD19,8.93,Non-Edible,0.009780,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train,26,Non-Consumables
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,NaN,test,16,Food
14200,FDD47,7.60,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,NaN,test,4,Food
14201,NCO17,10.00,Non-Edible,0.073529,Health and Hygiene,118.7440,OUT045,2002,Medium,Tier 2,Supermarket Type1,NaN,test,11,Non-Consumables
14202,FDJ26,15.30,Regular,0.088380,Canned,214.6218,OUT017,2007,Medium,Tier 2,Supermarket Type1,NaN,test,6,Food


## 3. Numerical and One Hot Encoding of categorical variables

### 3.1 Label Encoding

In [16]:
#import library
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#new variable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mode = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mode:
    data[i] = le.fit_transform(data[i])


### 3.2 One Hot Encoding

In [17]:
#one Hot Encoding
data = pd.get_dummies(data,columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet'])
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Outlet_0                       uint8
O

In [18]:
pd.set_option('max_columns',None)
data

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,14,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,4,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,14,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.017834,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,15,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.009780,Household,53.8614,OUT013,1987,994.7052,train,26,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,0.013496,Snack Foods,141.3154,OUT046,1997,NaN,test,16,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
14200,FDD47,7.60,0.142991,Starchy Foods,169.1448,OUT018,2009,NaN,test,4,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
14201,NCO17,10.00,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,test,11,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
14202,FDJ26,15.30,0.088380,Canned,214.6218,OUT017,2007,NaN,test,6,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


## 4 Exporting Data

In [19]:
#drop the data which has been converted into different types
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#divide the train and test data
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

#drop unnecesary columns
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export the modified version
train.to_csv("train_processed_data.csv",index=False)
test.to_csv("test_processed_data.csv",index=False)

